# Stage 1: Importing dependencies

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 41 kB 191 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=5730f0185c4ed0b328b68104e4d4e84cb50a753d8b1e1f68014896e1744ed120
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=8ed4df5c1ed501fb42869322dc6e688c6da203fc663267106e629baad7e68f75
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=ba3cc183edd09325b8d7291d9b6dda7274f485f61a52fc4c537e1bc023177a8c
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params
     |████████████████████████████████| 1.2 MB 7.7 MB/s 


In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

# Stage 2: Data preprocessing

## Loading files

We import files from our personal Google drive.

In [4]:
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/Datasets/SentimentData/training.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [6]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [7]:
data.head(5)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
data[data.sentiment == 4].head()

,sentiment,text
800000,4,I LOVE @Health4UandPets u guys r the best!!
800001,4,im meeting up with one of my besties tonight! ...
800002,4,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800003,4,Being sick can be really cheap when it hurts t...
800004,4,@LovesBrooklyn2 he has that effect on everyone


## Preprocessing

### Cleaning

In [9]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Delete the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Delete URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Just keep letters and important punctuation
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Remove additional spaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [10]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [11]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

In [12]:
data_clean[:10]

[" Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D",
 "is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!",
 ' I dived many times for the ball. Managed to save The rest go out of bounds',
 'my whole body feels itchy and like its on fire ',
 " no it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. ",
 ' not the whole crew ',
 'Need a hug ',
 " hey long time no see! Yes.. Rains a bit only a bit LOL I'm fine thanks how's you ?",
 " K nope they didn't have it ",
 ' que me muera ? ']

### Tokenization

We need to create a BERT layer to have access to meta data for the tokenizer (like vocab size).

In [13]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [14]:
print("lower case : {do_lower_case}")
print("vocab file : {vocab_file}")
with open(vocab_file, "r") as f:
  x = f.readlines()
  
  print("Vocab length: ", len(x))
  print("Some words: ", x[5000:5010])

  


lower case : {do_lower_case}
vocab file : {vocab_file}
Vocab length:  30522
Some words:  ['knight\n', 'lap\n', 'survey\n', 'ma\n', '##ow\n', 'noise\n', 'billy\n', '##ium\n', 'shooting\n', 'guide\n']


In [15]:
print(tokenizer.tokenize("what will be next variant after covid omicron?"))

['what', 'will', 'be', 'next', 'variant', 'after', 'co', '##vid', 'om', '##ic', '##ron', '?']


In [16]:
def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [17]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

### Dataset creation

We will create padded batches (so we pad sentences for each batch inpedendently), this way we add the minimum of padding tokens possible. For that, we sort sentences by length, apply padded_batches and then shuffle.

In [18]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [(sent_lab[0], sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [19]:
# A list is a type of iterator so it can be used as generator for a dataset
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [20]:
next(iter(all_dataset))

(<tf.Tensor: shape=(8,), dtype=int32, numpy=array([3225, 1996, 2154, 1012, 2154, 2147, 2733, 1012], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [21]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [22]:
next(iter(all_batched))

(<tf.Tensor: shape=(32, 8), dtype=int32, numpy=
 array([[ 3225,  1996,  2154,  1012,  2154,  2147,  2733,  1012],
        [ 1054,  1057,  1996,  2613,  9321, 22320,  5634,  1029],
        [ 4122,  2000,  2175,  2067,  2000,  5264,  1012,  1012],
        [ 2851,  3328,  1998,  2019,  5746,  8654,  2307,  2707],
        [ 2009,  1005,  1055,  2035,  2055, 26823,  9541,  7140],
        [ 1998,  2023,  2095,  1045,  2031,  2621,  2082,  1012],
        [22091,  1012,  1012,  1012,  3374,  2055,  7858,  1012],
        [ 8040,  3089, 11362,  2015,  2681,  2147,  2012,  2147],
        [ 2008,  2071,  2022,  3835,  2012,  1996,  2168,  2051],
        [ 1057,  3532,  2518,  1045,  2079,  2514,  2005,  2017],
        [ 2025,  3201,  2005,  1996,  5353,  2000,  2022,  2058],
        [ 2059,  2017,  1005,  1040,  2031, 12136, 20349,  2015],
        [ 4083,  1996,  7570, 26010,  7962,  5466,  7698,   999],
        [ 2047,  4231,  9117,  3310,  2041,  2101,  2651,   999],
        [ 2205,  2919,  1012

In [23]:
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Stage 3: Model building

In [24]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x) # (batch_size, nb_filters, seq_len-1)
        x_1 = self.pool(x_1) # (batch_size, nb_filters)
        x_2 = self.trigram(x) # (batch_size, nb_filters, seq_len-2)
        x_2 = self.pool(x_2) # (batch_size, nb_filters)
        x_3 = self.fourgram(x) # (batch_size, nb_filters, seq_len-3)
        x_3 = self.pool(x_3) # (batch_size, nb_filters)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Stage 4: Training

In [25]:
VOCAB_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 2

In [26]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [27]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [28]:
checkpoint_path = "/content/drive/MyDrive/BERT_UDEMY/models/cnn_sentiment/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest Checkpoint restored!")

Latest Checkpoint restored!


In [29]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [30]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])

Epoch 1/2
  37196/Unknown - 465s 12ms/step - loss: 0.2112 - accuracy: 0.9116Checkpoint saved at /content/drive/MyDrive/BERT_UDEMY/models/cnn_sentiment/.
37196/37196 [==============================] - 466s 12ms/step - loss: 0.2112 - accuracy: 0.9116
Epoch 2/2
37196/37196 [==============================] - 427s 11ms/step - loss: 0.1866 - accuracy: 0.9218


# Stage 5: Evaluation

In [31]:
results = Dcnn.evaluate(test_dataset)
print(results)

4132/4132 [==============================] - 27s 7ms/step - loss: 0.5449 - accuracy: 0.8271
[0.5449054837226868, 0.827111542224884]


In [38]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)
    inputs = tf.expand_dims(tokens, 0)

    output = Dcnn(inputs, training=False)

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Output of the model: {}\nPredicted sentiment: negative.".format(
            output))
    elif sentiment == 1:
        print("Output of the model: {}\nPredicted sentiment: positive.".format(
            output))

In [39]:
get_prediction("This movie was pretty interesting.")

Output of the model: [[0.97760576]]
Predicted sentiment: positive.


In [40]:
get_prediction("I'd rather not do that again.")

Output of the model: [[0.1810069]]
Predicted sentiment: negative.


In [58]:
get_prediction("phone is good but battery sucks.")

Output of the model: [[0.00339373]]
Predicted sentiment: negative.
